In [1]:
import requests
fsave='./HARP_TO_NOAA.txt'
def fetch_newest_harpnum_to_noaa(filename=fsave):
    '''
    This function downloads newest harp to noaa mapping from jsoc
    just call the function.
    '''
    url = 'http://jsoc.stanford.edu/doc/data/hmi/harpnum_to_noaa/all_harps_with_noaa_ars.txt'
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)

#fetch_newest_harpnum_to_noaa()

In [2]:
# Read content of HARPS TO NOAA in content
with open(fsave) as f:
    content = [line.rstrip() for line in f]
    
def harp_from_noaa(NOAANUM):
    '''
    This returns HARP number for provided NOAA region from content file
    '''
    ins = [index for index, string in enumerate(content) if NOAANUM in string]
    harpnum = [ content[x].split(' ')[0] for x in ins ]
    if len(harpnum) != 1:
        raise Exception("Be careful! Your region is over multiple HARPs or it was not found! HARPS: {}".format(harpnum))
    return harpnum[0]

In [3]:
HARPNUM=harp_from_noaa("11950")
MY_MAIL='lazar.zivadinovic.994@gmail.com'
print(HARPNUM)

3604


In [4]:
#Lets find primary keys and data sets for our set
import drms
c = drms.Client()
si = c.info('hmi.sharp_cea_720s')
# ['magnetogram' 'bitmap' 'Dopplergram' 'continuum' 'Bp' 'Bt' 'Br' 'Bp_err'
# 'Bt_err' 'Br_err' 'conf_disambig']

print(c.pkeys('hmi.sharp_cea_720s'))
# ['HARPNUM', 'T_REC']

# Lets make jsoc query 
from sunpy.net import jsoc
from sunpy.net import attrs as a
from sunpy.time import parse_time
#initialize client
client = jsoc.JSOCClient()
# Create query
resjsoc = client.search(a.jsoc.PrimeKey('HARPNUM', HARPNUM),
                        a.jsoc.PrimeKey('T_REC', '2014.01.08_03:12:00_TAI'),
                        #a.Time('2017-09-06T12:00:00', '2017-09-06T12:02:00'),
                        a.jsoc.Series('hmi.sharp_cea_720s'),
                        a.jsoc.Segment('Bp') & a.jsoc.Segment('magnetogram'), 
                        a.jsoc.Notify(MY_MAIL))


print(resjsoc)
print(resjsoc.query_args)
req = client.request_data(resjsoc)

save_path = './'
res = client.get_request(req, path=save_path,progress=True, wait=True)
print(res)
print(type(res))
print(dir(res))
#res.wait(progress=True)



['HARPNUM', 'T_REC']
         T_REC          TELESCOP  INSTRUME WAVELNTH CAR_ROT
----------------------- -------- --------- -------- -------
2014.01.08_03:12:00_TAI  SDO/HMI HMI_SIDE1   6173.0    2145
[{'primekey': {}, 'series': 'hmi.sharp_cea_720s', 'segment': ['Bp', 'magnetogram'], 'notify': 'lazar.zivadinovic.994@gmail.com'}]


ValueError: Atleast one PrimeKey must be passed.